# 임베딩 생성 및 벡터 저장 실습 (E5 임베딩 모델 사용)
- 사용 모델: intfloat/multilingual-e5-small
    - HuggingFace에서 공개한 다국어 임베딩 모델
    - 검색/질의 응답 최적화된 E5 구조
    - 참고 링크: https://huggingface.co/intfloat/multilingual-e5-small
- 실습 목표
    - 사전 학습된 E5 모델 로딩하기
    - 텍스트 데이터를 벡터(임베딩)로 변환
    - FAISS를 사용해 벡터 인덱스를 만들고 저장
    - 간단한 질의로 검색 테스트

# 1. 필수 라이브러리 설치 및 로드

In [1]:
# !pip install -U langchain faiss-cpu langchain-huggingface

In [2]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 2. E5 임베딩 모델 설명

임베딩은 텍스트를 고정 길이의 벡터로 바꾸는 과정으로, 의미 기반 검색(Retrieval)에 핵심적인 역할을 합니다.

E5 모델은 다음과 같은 특징을 가집니다:
- query/document 구조를 구분하여 학습된 RAG 특화 모델
- 다양한 언어(한국어 포함)를 지원하는 다국어 모델
- 문서 임베딩에 효과적이며, 성능 대비 속도도 우수
- 문서 검색, Q&A, Chat RAG 등 다양한 태스크에서 활용 가능

특히 "intfloat/multilingual-e5-small"은 가벼운 실습 및 개발용으로 최적화되어 있어,
실제 프로젝트 이전의 프로토타이핑에 매우 유용합니다.

# 3. 텍스트 로드 및 문단 단위 분할

In [3]:
# 텍스트 로드
file_path = "kr_constitution_cleaned.txt"

with open(file_path, "r", encoding="utf-8") as f:
    full_text = f.read()

텍스트 전체를 한 번에 임베딩하면 길이 제한(token limit)에 걸릴 수 있으므로, 적절히 분할해야 합니다. 이를 Chunking이라 하며 다음과 같은 방식이 있습니다.
- 줄바꿈 기준으로 문단 분할
- 일정 길이(예: 500자) 기준 슬라이딩 윈도우 방식
- 의미 단위로 분할 (문장, 문단 등)
이 예시에서는 가장 단순한 줄바꿈 기준 분할을 사용합니다.

In [4]:
chunks = full_text.split("\n")  # 혹은 500자 단위 슬라이딩 윈도우 등으로 수정 가능
chunks = [c.strip() for c in chunks if len(c.strip()) > 30]  # 너무 짧은 문장은 제외

print("총 문단 수:", len(chunks))
print("샘플 문단:", chunks[0])

총 문단 수: 261
샘플 문단: [시행 1988. 2. 25.] [헌법 제10호, 1987. 10. 29., 전부개정]


# 5. 임베딩 모델 로드 (E5)

In [5]:
model_name = "intfloat/multilingual-e5-small"
embedding = HuggingFaceEmbeddings(model_name=model_name)

/Users/jiho/Desktop/rag-chatbot-guide/rag-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 6. 텍스트 벡터화 및 FAISS 인덱스 생성
텍스트 분할 결과(chunks)를 임베딩 모델에 입력하여 벡터로 변환한 뒤, FAISS(Vector Store)에 저장합니다.

이 과정을 통해 나중에 사용자의 질문을 벡터화하여,벡터 간 유사도를 계산하고 관련 문서를 빠르게 검색할 수 있게 됩니다.

In [6]:
faiss_index = FAISS.from_texts(chunks, embedding)
faiss_index.save_local("embeddings/faiss_index_e5")

print("\n✅ FAISS 인덱스 저장 완료")


✅ FAISS 인덱스 저장 완료


# 6. 벡터 검색 테스트

In [7]:
retriever = faiss_index.as_retriever()

In [8]:
# 검색 옵션 설정: 상위 k개 문서 검색
k = 5
retriever.search_kwargs["k"] = k

query = "국회의 임기는 몇 년인가요?"
results = retriever.get_relevant_documents(query)

print("\n🔍 질문:", query)
for i, doc in enumerate(results):
    print(f"[{i+1}] {doc.page_content.strip()[:100]}...")

/var/folders/5q/7xd0r_rd1k76n08xqd5tfknm0000gn/T/ipykernel_14378/2092163242.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



🔍 질문: 국회의 임기는 몇 년인가요?
[1] ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다....
[2] 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최...
[3] ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다....
[4] 제86조 ①국무총리는 국회의 동의를 얻어 대통령이 임명한다....
[5] 제104조 ①대법원장은 국회의 동의를 얻어 대통령이 임명한다....
